# LightRAG usage example

## Imports

In [1]:

import os
import json
import logging
import base64
from getpass import getpass

import nest_asyncio
import networkx as nx
from pyvis.network import Network
from langchain_openai import ChatOpenAI
from huggingface_hub import notebook_login, login
from transformers import AutoModel, AutoTokenizer

from lightrag import LightRAG, QueryParam
from lightrag.llm import (
    hf_embedding,
    openai_complete_if_cache,

)
from lightrag.utils import EmbeddingFunc

from utils.config import Configuration

## Setting up LightRAG

### Config loading & workdir creation

In [2]:
if not os.path.exists("secrets.json"):
    secrets = {
        "samba_nova_api_key": [
            getpass(prompt="Your SambaNova API key (input hiddden): ")
        ],
        "huggingface_token": [
            getpass(prompt="Your Hugging Face token (input hidden): ")
        ],
    }
    with open("secrets.json", "w", encoding="utf-8") as f:
        json.dump(secrets, f, ensure_ascii=False, indent=4)
if not os.path.exists("config.json"):
    with open("config.json", "w", encoding="utf-8") as f:
        json.dump({}, f, ensure_ascii=False, indent=4)

config = Configuration("config.json")

try:
    token = config["huggingface_token"]
    assert isinstance(token, str), "Invalid token type encountered somehow"
    login(token)
    print("Login successful!")
except KeyError:
    notebook_login()

nest_asyncio.apply()
WORKING_DIR = config["processed/lightrag"]
DATA_PATH = config["raw/"]
assert isinstance(
    WORKING_DIR, str
), f"WORKING_DIR should be a string but is {type(WORKING_DIR)}"
assert isinstance(
    DATA_PATH, str
), f"DATA_PATH should be a string but is {type(DATA_PATH)}"
empty = not os.path.exists("lightrag") or not os.listdir("lightrag")
os.makedirs(WORKING_DIR, exist_ok=True)

Login successful!


### LightRAG init

In [3]:
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], **kwargs
) -> str:
    return await openai_complete_if_cache(
        "Meta-Llama-3.1-405B-Instruct",
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=config['samba_nova_api_key'],
        base_url="https://api.sambanova.ai/v1",
        **config['openai_complete_if_cache_kwargs']
    )


rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func,
    embedding_func=EmbeddingFunc(
        embedding_dim=384,
        max_token_size=5000,
        func=lambda texts: hf_embedding(
            texts,
            tokenizer=AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"),
            embed_model=AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        )
    ),
    **config['rag_kwargs']
)

INFO:lightrag:Logger initialized for working directory: data/processed/lightrag
INFO:lightrag:Load KV llm_response_cache with 0 data
INFO:lightrag:Load KV full_docs with 0 data
INFO:lightrag:Load KV text_chunks with 0 data
INFO:nano-vectordb:Init {'embedding_dim': 384, 'metric': 'cosine', 'storage_file': 'data/processed/lightrag/vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 384, 'metric': 'cosine', 'storage_file': 'data/processed/lightrag/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 384, 'metric': 'cosine', 'storage_file': 'data/processed/lightrag/vdb_chunks.json'} 0 data


### Inserting data

In [4]:
pdf_path = config['raw/Росконгресс_Рынок_промышленных_роботов_в_мире_и_России_2024_16_стр.pdf']
docx_path = config['raw/СП_496_1325800_2020_Основания_и_фундаменты_зданий_и_сооружений.docx']
rag.insert([pdf_path, docx_path], config=config)

INFO:lightrag:[New Docs] inserting 2 docs
Chunking documents:   0%|          | 0/2 [00:00<?, ?doc/s]

Extracting content from file: data/raw/Росконгресс_Рынок_промышленных_роботов_в_мире_и_России_2024_16_стр.pdf...done!


Chunking documents: 100%|██████████| 2/2 [00:03<00:00,  1.78s/doc]
INFO:lightrag:[New Chunks] inserting 217 chunks
INFO:lightrag:Inserting 217 vectors to chunks


Extracting content from file: data/raw/СП_496_1325800_2020_Основания_и_фундаменты_зданий_и_сооружений.docx...done!


Generating embeddings: 100%|██████████| 7/7 [00:36<00:00,  5.15s/batch]
INFO:lightrag:[Entity Extraction]...
Extracting entities from chunks:   0%|          | 0/217 [00:00<?, ?chunk/s]INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completio

...

Extracting entities from chunks:  98%|█████████▊| 213/217 [14:03<00:15,  3.87s/chunk]INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"


Extracting entities from chunks:  99%|█████████▊| 214/217 [14:06<00:10,  3.52s/chunk]INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"


Extracting entities from chunks:  99%|█████████▉| 215/217 [14:10<00:07,  3.51s/chunk]INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"


Extracting entities from chunks: 100%|█████████▉| 216/217 [14:22<00:06,  6.32s/chunk]INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"


Extracting entities from chunks: 100%|██████████| 217/217 [14:28<00:00,  4.00s/chunk]
INFO:lightrag:Inserting entities into storage...
Inserting entities: 100%|█████████▉| 1602/1605 [00:05<00:00, 286.73entity/s]INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
Inserting entities: 100%|██████████| 1605/1605 [00:06<00:00, 234.22entity/s]
INFO:lightrag:Inserting relationships into storage...
Inserting relationships: 100%|██████████| 1179/1179 [00:00<00:00, 9337.59relationship/s]
INFO:lightrag:Inserting 1605 vectors to entities
Generating embeddings: 100%|██████████| 51/51 [03:35<00:00,  4.22s/batch]  
INFO:lightrag:Inserting 1179 vectors to relationships
Generating embeddings: 100%|██████████| 37/37 [02:02<00:00,  3.30s/batch]  
INFO:lightrag:Writing graph with 16

## QA test

In [5]:
single = ChatOpenAI(api_key=config['samba_nova_api_key'],
                base_url="https://api.sambanova.ai/v1",
                model="Meta-Llama-3.1-405B-Instruct",
                streaming=True)

In [6]:
query = "В каких случаях применяют повышение кровли ММГ до уровня подошвы насыпи для производства земляных работ при устройстве оснований и фундаментов на ММГ"
print('\n## RAG-Answer\n')
print(rag.query(query, param=QueryParam(mode="local",
                                                                            max_token_for_global_context=1650,
                                                                            max_token_for_local_context=1650,
                                                                            max_token_for_text_unit=1450)))
print('\n## -Llama-405B-Answer\n')

print(single.invoke(query).content)


## RAG-Answer



INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


Since the query is in Russian, I will provide the output in English, as per the instructions.

Query Translation: "In what cases is the roof of the MMS raised to the level of the base of the embankment for earthworks during the construction of foundations and foundations on the MMS?"

Output:
{
  "high_level_keywords": ["Earthworks", "Foundation construction", "MMS ( Mechanized Modular System )"],
  "low_level_keywords": ["Roof raising", "Embankment base", "Soil excavation", "Foundation installation", "Modular construction"]
}


INFO:lightrag:Local query uses 60 entites, 74 relations, 5 text units
INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"


### Повышение кровли ММГ до уровня подошвы насыпи

Повышение кровли ММГ (мерзлотно-мёрзлых грунтов) до уровня подошвы насыпи является одним из методов производства земляных работ при устройстве оснований и фундаментов на ММГ. Этот метод применяется в следующих случаях:

*   **При значительном превышении уровня подошвы насыпи над кровлей ММГ**: В этом случае повышение кровли ММГ позволяет уменьшить объём земляных работ и сократить время производства работ.
*   **При необходимости устройства фундаментов на ММГ с низкой несущей способностью**: Повышение кровли ММГ позволяет увеличить несущую способность грунта и обеспечить устойчивость фундаментов.
*   **При наличии подземных вод или агрессивных грунтов**: Повышение кровли ММГ позволяет уменьшить влияние подземных вод или агрессивных грунтов на фундамент и обеспечить его долговечность.

### Технология повышения кровли ММГ

Технология повышения кровли ММГ включает в себя следующие этапы:

1.  **Подготовка территории**: Перед началом работ 

INFO:httpx:HTTP Request: POST https://api.sambanova.ai/v1/chat/completions "HTTP/1.1 200 OK"


Повышение кровли ММГ (массива мерзлых грунтов) до уровня подошвы насыпи применяется в следующих случаях:

1. **При значительной толщине слоя мерзлых грунтов**: Если толщина слоя мерзлых грунтов велика, то повышение кровли ММГ до уровня подошвы насыпи позволяет уменьшить объем земляных работ и сократить время производства работ.
2. **При высоком уровне грунтовых вод**: Если уровень грунтовых вод высок, то повышение кровли ММГ до уровня подошвы насыпи позволяет предотвратить затопление котлована или траншеи и облегчить производство земляных работ.
3. **При неустойчивых грунтах**: Если грунты неустойчивы и склонны к оползням или обрушениям, то повышение кровли ММГ до уровня подошвы насыпи позволяет повысить устойчивость откосов и обеспечить безопасность производства работ.
4. **При необходимости устройства глубоких фундаментов**: Если необходимо устройство глубоких фундаментов, то повышение кровли ММГ до уровня подошвы насыпи позволяет уменьшить глубину котлована или траншеи и сократить в

## Miscellaneous

### Vision LLM test

In [ ]:
vision = ChatOpenAI(api_key=config['samba_nova_api_key'],
                base_url="https://api.sambanova.ai/v1",
                model="Llama-3.2-11B-Vision-Instruct",
                streaming=True)

In [ ]:
image_path = config['raw/image.jpg']
with open(image_path, "rb") as image_file:
    encoded = base64.b64encode(image_file.read())
    url = f'data:image/jpeg;base64,{encoded.decode("utf-8")}'
print(vision.invoke([{"role":"user","content":[{"type":"text","text":"What do you see in this image"},{"type":"image_url","image_url":{"url":url}}]}]).content)

### Saving the knowledge graph

In [ ]:

# Load the GraphML file
G = nx.read_graphml(f'{WORKING_DIR}/graph_chunk_entity_relation.graphml')

# Create a Pyvis network
net = Network(notebook=True)

# Convert NetworkX graph to Pyvis network
net.from_nx(G)

# Save and display the network
net.save_graph(f'{WORKING_DIR}/knowledge_graph.html')

In [ ]:
G